In [1]:
!pip -q install transformers==4.17
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 848.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installa

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import time

**Dataset Utlilization**

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())
print(train_df.shape)
print(test_df.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


(1884, 6)
(406, 6)


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-32a081f8d74d>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-32a081f8d74d>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=5)

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/59.8M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        # Log training loss
        if self.state.global_step % self.args.logging_steps == 0:
            print(f"Step {self.state.global_step}, Training Loss: {loss.item()}")

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)
    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
# def objective(trial):
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
#     batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 20)
#     weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size * 2,
#         warmup_steps=10,
#         weight_decay=weight_decay,
#         evaluation_strategy="epoch",
#         logging_dir="./logs",
#         logging_steps=10,
#         save_steps=500,
#         learning_rate=learning_rate,
#         gradient_accumulation_steps=2,
#         report_to="none",
#         logging_first_step=True,
#     )

#     trainer = OneHotTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         compute_metrics=compute_metrics,
#     )
#     print("Starting training...")
#     trainer.train()
#     eval_results = trainer.evaluate()

#     return eval_results["eval_f1"]

In [11]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=20)

# print("Best trial:")
# print(study.best_trial)

In [12]:
training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    learning_rate=3.1111e-5,
    weight_decay=0.02,
    warmup_steps=500,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    model=model,
    compute_metrics=compute_metrics,
)

# Measure training time
print("Starting training...")
start_time = time.time()  # Start the timer

trainer.train()

end_time = time.time()  # End the timer
training_duration = end_time - start_time  # Calculate the duration

# Print the training duration
print(f"Training completed in {training_duration:.2f} seconds.")

Starting training...


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 7540


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.509916,0.000000,1.000000,0.000000,0.000000
2,0.574600,0.494700,0.034483,0.947021,0.031941,0.056080
3,0.483800,0.443000,0.299735,0.869631,0.292383,0.341743
4,0.409000,0.403160,0.416446,0.846805,0.407862,0.413092
5,0.409000,0.375918,0.493369,0.841003,0.486486,0.513505
6,0.335000,0.365387,0.511936,0.840398,0.501229,0.520999
7,0.281700,0.379874,0.543767,0.731531,0.533170,0.543065
8,0.244400,0.349464,0.612732,0.747461,0.609337,0.620002
9,0.244400,0.369702,0.604775,0.698910,0.616708,0.622672
10,0.207400,0.366920,0.610080,0.681931,0.631450,0.637772


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./best_results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./best_results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./best_results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./best_results/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1500
Configuratio

Training completed in 296.03 seconds.


In [13]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


### Model Saving

In [14]:
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_python'
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_huawei-noah_TinyBERT_General_4L_312D_python')
tokenizer.save_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python')
tokenizer = AutoTokenizer.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer')

Saving model checkpoint to ./best_model_huawei-noah_TinyBERT_General_4L_312D_python
Configuration saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/config.json
Model weights saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/pytorch_model.bin
tokenizer config file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/special_tokens_map.json
tokenizer config file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_huawei-noah_TinyBERT_General_4L_312D_python",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.5221674876847291, 'precision': 0.6030130163785491, 'recall': 0.5619266055045872, 'f1': 0.5737822778736077}
Inference Time: 0.96 seconds


In [18]:
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_python'
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/pytorch_model.bin (deflated 7%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/tokenizer_config.json (deflated 41%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/vocab.txt (deflated 53%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/special_tokens_map.json (deflated 40%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/training_args.bin (deflated 51%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/config.json (deflated 52%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python/tokenizer.json (deflated 71%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer/tokenizer_config.json (deflated 41%)
  adding: best_model_huawei-noah_TinyBERT_Ge

In [19]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip "/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python/"
!cp best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip "/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_folder = '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python/'
tokenizer_folder = '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip' -d './'
!unzip '/content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip
replace ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/pytorch_model.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/tokenizer_config.json  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/vocab.txt  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/special_tokens_map.json  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/training_args.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/config.json  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_python/tokenizer.json  
Archive:  /content/drive/MyDrive/nlbse/models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best

In [22]:
best_model_huawei_noah_TinyBERT_General_4L_312D_python = 'best_model_huawei-noah_TinyBERT_General_4L_312D_python'
best_model_huawei_noah_TinyBERT_General_4L_312D_python_tokenizer = 'best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

In [23]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_python)
tokenizer = AutoTokenizer.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_python_tokenizer)

loading configuration file best_model_huawei-noah_TinyBERT_General_4L_312D_python/config.json
Model config BertConfig {
  "_name_or_path": "best_model_huawei-noah_TinyBERT_General_4L_312D_python",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "multi_label_classification",
  "s

In [24]:
text = "for detecting automatically generated fields."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.9134, -5.5075,  0.2711, -1.5928, -1.7024]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 1, 3
